In [ ]:
import boto3
import pandas as pd
from currency_converter import CurrencyConverter
import numpy as np
from src.utils import google
from typing import Iterator
import os
import dotenv
dotenv.load_dotenv()

In [ ]:
SMART_MONEY_TYPES = ['accelerator', 'entrepreneurship_program', 'incubator', 'startup_competition']

In [ ]:
cb_organisations = pd.read_parquet('organizations.parquet')
funding_rounds = pd.read_parquet('funding_rounds.parquet')
investments = pd.read_parquet('investments.parquet')
investors = pd.read_parquet('investors.parquet')


## Wrangler code

In [ ]:
# class CrunchbaseWrangler:
#     """
#     This class helps linking CB companies to other data such as the
#     investment rounds (deals) and investors participating in the deals
#     """

#     def __init__(self, cb_data_path: str = None):
#         """
#         Sets up the CrunchbaseWrangler class

#         Args:
#             cb_data_path: Optional argument to specify the location of the
#                 Crunchbase data, if it is different from the default location;
#                 NB: Note that this changes a global variable.
#         """
#         # Tables from the database
#         self._cb_organisations = None
#         self._cb_funding_rounds = None
#         self._cb_investments = None
#         self._cb_investors = None
#         self._cb_category_groups = None
#         self._cb_organisation_categories = None
#         self._cb_people = None
#         self._cb_degrees = None
#         # Organisation categories (industries)
#         self._industries = None
#         self._industry_groups = None
#         self._industry_to_group = None
#         self._group_to_industries = None
#         # Set data path (optional)
#         if cb_data_path is not None:
#             cb.CB_PATH = cb_data_path

#     @property
#     def cb_organisations(self):
#         """Full table of companies (this might take a minute to load in)"""
#         if self._cb_organisations is None:
#             self._cb_organisations = cb.get_crunchbase_orgs()
#         return self._cb_organisations

#     @property
#     def cb_funding_rounds(self):
#         """Table with investment rounds (all the deals for all companies)"""
#         if self._cb_funding_rounds is None:
#             self._cb_funding_rounds = cb.get_crunchbase_funding_rounds()
#         return self._cb_funding_rounds

#     @property
#     def cb_investments(self):
#         """
#         Table with investments for each round (deal). Note that one deal can
#         have several investments pertaining to different investors
#         """
#         if self._cb_investments is None:
#             self._cb_investments = cb.get_crunchbase_investments()
#         return self._cb_investments

#     @property
#     def cb_investors(self):
#         """Table with investors"""
#         if self._cb_investors is None:
#             self._cb_investors = cb.get_crunchbase_investors()
#         return self._cb_investors

#     def get_funding_round_investors(self, funding_rounds: pd.DataFrame) -> pd.DataFrame:
#         """
#         Gets the investors involved in the specified funding rounds

#         Args:
#             Dataframe with organisations specified by 'org_id' and 'name', and
#             all their funding rounds (deals).

#         Returns:
#             Dataframe with extra columns specifying investment round details:
#                 'funding_round_name': Name of the funding round
#                 'investor_name': Name of the investor organisation
#                 'investor_id' Crunchbase organisation identifier
#                 'investor_type': Specifies if investor is a person or an organisation
#                 'is_lead_investor': Specifies whether the organisation is leading the round (value is 1 in that case)

#         """
#         return funding_rounds.merge(
#             (
#                 self.cb_investments[
#                     [
#                         "funding_round_id",
#                         "funding_round_name",
#                         "investor_name",
#                         "id",
#                         "investor_type",
#                         "is_lead_investor",
#                     ]
#                 ].rename(columns={"id": "investor_id"})
#             ),
#             on="funding_round_id",
#             how="left",
#         ).assign(
#             raised_amount=lambda x: x.raised_amount.fillna(0),
#             raised_amount_usd=lambda x: x.raised_amount_usd.fillna(0),
#         )

#     def get_organisation_investors(
#         self, cb_organisations: pd.DataFrame
#     ) -> pd.DataFrame:
#         """
#         Gets the investors and corresponding funding rounds for the specified organisations

#         Args:
#             cb_organisations: Data frame that must have a columns with crunchbase
#                 organisation ids and names

#         Returns:
#              Dataframe with extra columns specifying investors and the corresponding investment rounds
#              (see docs for get_funding_rounds and get_funding_round_investors for more information)

#         """
#         return pipe(
#             cb_organisations, self.get_funding_rounds, self.get_funding_round_investors
#         )

    


## Enrichment utils

In [ ]:
def convert_currency(
    funding_df: pd.DataFrame,
    date_column: str,
    amount_column: str,
    currency_column: str,
    converted_column: str = None,
    target_currency: str = "GBP",
) -> pd.DataFrame:
    """
    Convert amount in any currency to a target currency using CurrencyConverter package.
    Deal dates should be provided in the datetime.date format
    NB: Rate conversion for dates before year 2000 is not reliable and hence
    is not carried out (the function returns nulls instead)

    Args:
        funding_df: A dataframe which must have a column for a date and amount to be converted
        date_column: Name of column with deal dates
        amount_column: Name of column with the amounts in the original currency
        currency_column: Name of column with the currency codes (eg, 'USD', 'EUR' etc)
        converted_column: Name for new column with the converted amounts

    Returns:
        Same dataframe with an extra column for the converted amount

    """
    # Column name
    converted_column = (
        f"{amount_column}_{target_currency}"
        if converted_column is None
        else converted_column
    )
    # Check if there is anything to convert
    rounds_with_funding = len(funding_df[-funding_df[amount_column].isnull()])
    df = funding_df.copy()
    if rounds_with_funding > 0:
        # Set up the currency converter
        Converter = CurrencyConverter(
            fallback_on_missing_rate=True,
            fallback_on_missing_rate_method="linear_interpolation",
            # If the date is out of bounds (eg, too recent)
            # then use the closest date available
            fallback_on_wrong_date=True,
        )
        # Convert currencies
        converted_amounts = []
        for _, row in df.iterrows():
            # Only convert deals after year 1999
            if (row[date_column].year >= 2000) and (
                row[currency_column] in Converter.currencies
            ):
                converted_amounts.append(
                    Converter.convert(
                        row[amount_column],
                        row[currency_column],
                        target_currency,
                        date=row[date_column],
                    )
                )
            else:
                converted_amounts.append(np.nan)
        df[converted_column] = converted_amounts
        # For deals that were originally in the target currency, use the database values
        deals_in_target_currency = df[currency_column] == target_currency
        df.loc[deals_in_target_currency, converted_column] = df.loc[
            deals_in_target_currency, amount_column
        ].copy()
    else:
        # If nothing to convert, copy the values and return
        df[converted_column] = df[amount_column].copy()
    return df

def get_org_funding_rounds(
    organisations: pd.DataFrame,
    funding_rounds: pd.DataFrame,
    org_id_column: str = 'id'
) -> pd.DataFrame:
    """
    Get the funding rounds for a specific organisation

    Args:
        org_id: Crunchbase organisation identifier
        funding_rounds: Dataframe with funding rounds
        investments: Dataframe with investments
        investors: Dataframe with investors

    Returns:
        Dataframe with the funding rounds for the specified organisation
    """
    # Get the funding rounds for the organisation
    return (
        organisations[[org_id_column, "name"]]
        .rename(columns={org_id_column: "org_id"})
        .merge(
            funding_rounds,
            how = 'left',
            on = 'org_id'
        )
    )

def get_funding_round_investors(funding_rounds: pd.DataFrame, investments, investors) -> pd.DataFrame:
    """
    Gets the investors involved in the specified funding rounds

    Args:
        Dataframe with organisations specified by 'org_id' and 'name', and
        all their funding rounds (deals).

    Returns:
        Dataframe with extra columns specifying investment round details:
            'funding_round_name': Name of the funding round
            'investor_name': Name of the investor organisation
            'investor_id' Crunchbase organisation identifier
            'investor_type': Specifies if investor is a person or an organisation
            'is_lead_investor': Specifies whether the organisation is leading the round (value is 1 in that case)

    """
    investments_cols = [
        "funding_round_id",
        "investor_id",
        "investor_name",
        "investor_type",
        "is_lead_investor",
        ]
    investor_cols = [
        "id",
        "investor_types",
        "cb_url",
    ]
    return (
        funding_rounds
        .merge(
            investments[investments_cols],
            on="funding_round_id",
            how="left",
        )
        .merge(
            investors[investor_cols].rename(columns={"id": "investor_id", "cb_url": "investor_url"}),
            on="investor_id",
            how="left",
        )
    )

In [ ]:
def _enrich_smart_money(
    funding_rounds_enriched: pd.DataFrame,
    investments: pd.DataFrame,
    investors: pd.DataFrame,
) -> pd.DataFrame:
    """
    """
    # Fetch the manually curated smart money table
    smart_money_manual_df = google.access_google_sheet(
        os.environ['SHEET_ID_INVESTORS'],
        "investors"
    )
    return (
        funding_rounds_enriched
        .pipe(get_funding_round_investors, investments=investments, investors=investors)
        .assign(            
            # Check if investment was made by one of the smart money types
            smart_money_auto = lambda df: df.investor_types.str.contains('|'.join(SMART_MONEY_TYPES)),
            # Check if investment was made by one of the manually curated smart money investors
            smart_money_manual = lambda df: df.investor_url.isin(smart_money_manual_df['Crunchbase URL']),
            # Combine the two smart money columns
            smart_money = lambda df: df.smart_money_auto | df.smart_money_manual
        )
    )

def enrich_funding_rounds(
    funding_rounds: pd.DataFrame,
    investments: pd.DataFrame,
    investors: pd.DataFrame,
    funding_round_ids: Iterator[str] = None, 
    cutoff_year: int = 2000,
) -> pd.DataFrame:
    """
    Add funding round information to a table with CB organisations

    Args:
        cb_organisations: Data frame that must have a columns with crunchbase
            organisation ids and names

    Returns:
        Dataframe with organisations specified by 'org_id' and 'name', and
        all their funding rounds (deals). Some of the important columns include:
            'funding_round_id': unique round id
            'announced_on': date of the round
            'investment_type': the type of round (seed, series etc.)
            'post_money_valuation_usd': valuation of the company
            'raised_amount_usd': amount of money raised in the deal (in thousands)

    """
    # If no IDs are specified, assume all funding rounds are needed
    if funding_round_ids is None:
        funding_round_ids = funding_rounds.id.unique()

    return (
        funding_rounds.query("id in @funding_round_ids")
        # More informative column names
        .rename(columns={
            "id": "funding_round_id",
            "name": "funding_round_name",
        })
        # Remove really old funding rounds
        .query(f"announced_on > '{cutoff_year}'")
        .sort_values("announced_on")
        .assign(
            # Convert investment amounts to thousands
            raised_amount=lambda df: df["raised_amount"] / 1e+3,
            raised_amount_usd=lambda df: df["raised_amount_usd"] / 1e+3,
            # Convert date strings to datetimes
            announced_on_date=lambda df: pd.to_datetime(df["announced_on"]),
        )
        # Get years from dates
        .assign(year=lambda df: df["announced_on_date"].apply(lambda x: x.year))
        # Convert USD currency to GBP
        .pipe(
            convert_currency,
            date_column="announced_on_date",
            amount_column="raised_amount",
            currency_column="raised_amount_currency_code",
            converted_column="raised_amount_gbp",
            target_currency="GBP",
        )
        .pipe(
            _enrich_smart_money,
            investments=investments,
            investors=investors,
        )
    )


def _enrich_total_funding_gbp(
    organisations: pd.DataFrame,
    funding_rounds_enriched: pd.DataFrame,
    date_start: int = None,
    date_end:  str = None,
):
    """
    Get the total funding in GBP for a set of organisations

    Args:
        funding_rounds_enriched_df: Dataframe with enriched funding rounds
        date_start: Earliest date for the funding rounds to include, in the format 'YYYY-MM-DD'
        date_end: Latest date for the funding rounds to include, in the format 'YYYY-MM-DD'
    
    Returns:
        Dataframe with columns 'org_id' and 'total_funding_gbp'. Organisations that have no funding
        in the specified time period will not be included in the dataframe
    """
    # If no IDs are specified, assume all organisations are needed
    organisation_ids = funding_rounds_enriched.org_id.unique()

    # Determine the dates to include
    date_start = date_start or funding_rounds_enriched.announced_on.min()
    date_end = date_end or funding_rounds_enriched.announced_on.max()

    funding_rounds_df = (
        funding_rounds_enriched
        .loc[funding_rounds_enriched['announced_on'].between(date_start, date_end, inclusive='both')]
        .query("org_id in @organisation_ids")
        .groupby("org_id")
        .agg(
            total_funding_gbp=("raised_amount_gbp", "sum"),
            num_rounds=("funding_round_id", "count"),
        )
        .reindex(organisation_ids)
        .reset_index()
        .fillna(0)        
    )

    return (
        organisations
        .merge(
            funding_rounds_df,
            how = 'left',
            left_on='id',
            right_on='org_id'
        )
        .drop(columns='org_id')
    )   


In [ ]:
def _process_max_employees(count_range: str) -> int:
    """
    Convert the employee count range to a single (highest) number

    Args:
        count_range: A string with the employee count range, eg, '51-100', '10000+'

    Returns:
        A single integer with the maximum number of employees
    """
    if count_range is None:
        return 0
    elif type(count_range) == str:
        return int(count_range.split('-')[-1].replace('+', ''))
    else:
        raise ValueError(f"Unexpected value for employee count: {count_range}")


def _enrich_investment_opportunity(
    organisations_enriched: pd.DataFrame
) -> pd.DataFrame:
    """
    """
    # Check for valid employee count and country
    valid_count_and_country = (
        (organisations_enriched.employee_count_max <= 100) 
        & (organisations_enriched.country_code == 'GBR')         
    )
    # Investment opportunities tags
    potential_investment_opp = (
        valid_count_and_country
        & (organisations_enriched.total_funding_gbp < 1e+3)
    )
    investment_opp = (
        valid_count_and_country
        & (organisations_enriched.total_funding_gbp >= 1e+3)
        & (organisations_enriched.total_funding_gbp <= 5e+3)
    )
    return (
        organisations_enriched
        .assign(
            potential_investment_opp = potential_investment_opp,
            investment_opp = investment_opp
        )
    )    


In [ ]:
def enrich_organisations(
    organisations: pd.DataFrame,
    funding_rounds_enriched: pd.DataFrame,
    organisation_ids: Iterator[str] = None,
) -> pd.DataFrame:
    """
    """
     # If no IDs are specified, assume all organisations are needed
    organisation_ids = organisation_ids or organisations.id.unique()

    organisations_enriched = (
        organisations
        .query("id in @organisation_ids")
        .assign(employee_count_max = lambda df: df['employee_count'].apply(_process_max_employees))
        .pipe(
            _enrich_total_funding_gbp,
            funding_rounds_enriched=funding_rounds_enriched
        )
        .pipe(_enrich_investment_opportunity)
        .pipe(_enrich_org_smart_money, funding_rounds_enriched=funding_rounds_enriched)
    )
    return organisations_enriched


def _enrich_org_smart_money(
    organisations: pd.DataFrame,
    funding_rounds_enriched: pd.DataFrame,
) -> pd.DataFrame:
    """
    """
    # find orgs that have received smart money
    smart_money_orgs = (
        funding_rounds_enriched
        .query("smart_money")
        .org_id
        .unique()
    )
    return organisations.assign(smart_money = lambda df: df.id.isin(smart_money_orgs))


In [ ]:
# Get funding rounds enriched
funding_rounds_enriched_df = enrich_funding_rounds(funding_rounds, investments, investors)

In [ ]:
df = _enrich_smart_money(funding_rounds_enriched_df, investments, investors)
# rounds = get_org_funding_rounds(cb_organisations.head(100), funding_rounds_enriched_df)
# get_funding_round_investors(rounds, investments, investors)

In [ ]:
orgs_enriched = enrich_organisations(cb_organisations, funding_rounds_enriched_df)

In [ ]:
check = (
    orgs_enriched
    .query("smart_money == True")
    .query("investment_opp == True or potential_investment_opp == True")
)

In [ ]:
check

In [ ]:
# google.access_google_sheet(
#     "1-Sl2kglCa09r2Lg9CsGEKocc3m8pukOwwrbkQk7706I",
#     "ASF",
# )